In [2]:
import numpy as np
import pandas as pd

import time

import matplotlib.pyplot as plt

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials 
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id='99e688f0fa4a458da507b2625bd2201e', client_secret='2a1a63fa783f4ee5990d5283f77150c4'))

#uncomment above and replace CLIENT_ID and CLIENT_SECRET with the id and secret you get from creating your app with spotify (going through the "Getting Started" instructions in the Spotify Web API documentation)

from sklearn.model_selection import train_test_split
import math

# Spotify Integration

In [3]:
discussions_df = pd.read_csv('Discussions.csv', header=0, encoding='utf-8')
ratings_df = pd.read_csv('Ratings.csv', header=0, encoding='utf-8')

# creating new columns and intializing each value to 0
discussions_df['EnergyAvg'] = 0
discussions_df['ValenceAvg'] = 0
discussions_df['AcousticnessAvg'] = 0
discussions_df['InstrumentalnessAvg'] = 0

display(discussions_df)

,DiscussionID,AlbumName,ArtistName,Date,AvgRating,Stdev,Attendance,RotationGenre,OtherGenre,Subgenres,ReleaseYear,FavoriteTrack,Popularity,Tracks,SpotifyID,Image,EnergyAvg,ValenceAvg,AcousticnessAvg,InstrumentalnessAvg
0,1.0,IGOR,"Tyler, the Creator",2022-02-19,0.00,0.00,0.0,NaN,False,"Neo-Soul, Psychedelic Soul",2019.0,NaN,85.0,12.0,5zi7WsKlIiUXv09tbGLKsE,https://i.scdn.co/image/ab67616d0000b273700588...,0,0,0,0
1,2.0,The New Abnormal,The Strokes,2022-02-26,0.00,0.00,0.0,NaN,False,Indie Rock,2020.0,NaN,76.0,9.0,2xkZV2Hl1Omi8rk2D7t5lN,https://i.scdn.co/image/ab67616d0000b273bfa99a...,0,0,0,0
2,3.0,Blonde,Frank Ocean,2022-03-06,0.00,0.00,0.0,NaN,False,"Alternative R&B, Art Pop, Neo-Soul",2016.0,NaN,86.0,17.0,3mH6qwIy9crq0I9YQbOuDf,https://i.scdn.co/image/ab67616d0000b273c5649a...,0,0,0,0
3,4.0,So Tonight that I Might See,Mazzy Star,2022-03-27,0.00,0.00,0.0,NaN,False,"Dream Pop, Neo-Psychedelia",1993.0,NaN,73.0,10.0,5K18gTgac0q6Jma5HkV1vA,https://i.scdn.co/image/ab67616d0000b27389a392...,0,0,0,0
4,5.0,A New World Record,Electric Light Orchestra,2022-04-03,0.00,0.00,0.0,NaN,False,"Symphonic Rock, Pop Rock, Progressive Pop",1976.0,NaN,67.0,9.0,7a35UzxXYuKQGMGImyB0Un,https://i.scdn.co/image/ab67616d0000b273ee5b10...,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,67.0,Titanic Rising,Weyes Blood,2024-02-05,8.20,1.50,55.0,Pop,False,"Baroque Pop, Art Pop, Progressive Pop, Singer-...",2019.0,Andromeda,68.0,10.0,0Cuqhgy8vm96JEkBY3polk,https://i.scdn.co/image/ab67616d00001e020c64e7...,0,0,0,0
67,68.0,Because the Internet,Childish Gambino,2024-02-12,6.88,1.93,42.0,Hip-Hop,False,"Abstract Hip Hop, West Coast Hip Hop, Contempo...",2013.0,3005,81.0,19.0,62yjWIhnATHxPqGbgC9Lqr,https://i.scdn.co/image/ab67616d00001e0226d64b...,0,0,0,0
68,69.0,The Four Seasons,Antonio Vivaldi,2024-02-19,8.08,1.25,38.0,Classical,True,"Baroque Music, Concerto",1985.0,Winter: I. Allegro non molto,63.0,12.0,4eEJooUkzenp7lZTtfbIP7,https://i.scdn.co/image/ab67616d00001e02c7464f...,0,0,0,0
69,70.0,House Of Balloons (Original),The Weeknd,2024-02-26,8.04,1.08,30.0,Mainstream,True,"Alternative R&B, Downtempo",2011.0,House Of Balloons / Glass Table Girls,83.0,9.0,7zCODUHkfuRxsUjtuzNqbd,https://i.scdn.co/image/ab67616d00001e02274b40...,0,0,0,0


In [15]:
for ind in range(len(discussions_df)):
    results = spotify.album_tracks(discussions_df['SpotifyID'][ind]) # getting tracks from current album via SpotiPy

    # filtering track ids into list
    ids = []
    for track in results['items']:
        ids.append(track['id'])
    numTracks = len(results['items'])

    # getting features from each track in ids via SpotiPy
    features = spotify.audio_features(ids)
    
#     for i in range(len(features)):
#         # calculating and storing average in df
#         discussions_df['EnergyAvg'][ind] += (feature[i]['energy'] / numTracks)
#         discussions_df['ValenceAvg'][ind] += (feature[i]['valence'] / numTracks)
#         discussions_df['AcousticnessAvg'][ind] += (feature['acousticness'] / numTracks)
#         discussions_df['InstrumentalnessAvg'][ind] += (feature['instrumentalness'] / numTracks)
#     time.sleep(3) # sleep with arbitrary time of 3 seconds for circumventing rate limit
    
display(discussions_df)

TypeError: 'NoneType' object is not subscriptable

In [ ]:
discussions_df.to_csv("Discussions_Audio_Features.csv")

In [ ]:
discussionsAF_df = pd.read_csv('data/Discussions_Audio_Features.csv', header=0, encoding='utf-8')

ratings_df['WeightedEnergyRating'] = 0
ratings_df['WeightedValenceRating'] = 0
ratings_df['WeightedAcousticnessRating'] = 0
ratings_df['WeightedInstrumentalnessRating'] = 0

display(ratings_df)

In [ ]:
merged_df = pd.merge(discussionsAF_df, ratings_df, on='DiscussionID')
merged_df = merged_df.drop(['AlbumName', 'ArtistName', 'Date', 'AvgRating', 'Stdev', 'Attendance', 'RotationGenre', 'OtherGenre','Subgenres','ReleaseYear','FavoriteTrack','Popularity','Tracks','SpotifyID','Image', 'Unnamed: 0', 'FavoriteTrack1', 'FavoriteTrack2', 'FavoriteTrack3'], axis='columns') #cleaning up df

merged_df['WeightedEnergyRating'] = merged_df['Rating'] * merged_df['EnergyAvg']
merged_df['WeightedValenceRating'] = merged_df['Rating'] * merged_df['ValenceAvg']
merged_df['WeightedAcousticnessRating'] = merged_df['Rating'] * merged_df['AcousticnessAvg']
merged_df['WeightedInstrumentalnessRating'] = merged_df['Rating'] * merged_df['InstrumentalnessAvg']
display(merged_df)

# Baseline Estimate with weighted ratings

In [ ]:
# Standard mu + b_i + b_u
for AF in ['WeightedEnergyRating', 'WeightedValenceRating', 'WeightedAcousticnessRating', 'WeightedInstrumentalnessRating']:
    X = merged_df.drop(columns=[AF])
    y = merged_df[AF]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    X_train[AF] = y_train

    mu = np.mean(X_train[AF])
    y_pred = []
    for idx, row in X_test.iterrows():
        b_u = np.mean(X_train[X_train['MemberID'] == row['MemberID']][AF]) - mu
        b_u = 0 if math.isnan(b_u) else b_u
        b_i = np.mean(X_train[X_train['DiscussionID'] == row['DiscussionID']][AF]) - mu
        b_i = 0 if math.isnan(b_i) else b_i
        estimate = max(min(int(np.round(mu - b_u - b_i)), 10), 1)
        y_pred.append(estimate)
    y_pred = np.array(y_pred)
    rmse = np.sqrt(np.mean((y_pred - y_test)**2))
    print(AF + " rmse: " + str(rmse))

# User-User Collaborative Filtering with weighted Audio Feature Ratings

In [48]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    retValue = dot_product / (norm_vec1 * norm_vec2) if not math.isnan(dot_product / (norm_vec1 * norm_vec2)) else 0
    return retValue

def count_non_zeros(arr):
    return sum(1 for num in arr if num != 0)

In [49]:
y_pred = []

X = merged_df.drop(columns=['WeightedInstrumentalnessRating'])
y = merged_df['WeightedInstrumentalnessRating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train['WeightedInstrumentalnessRating'] = y_train

pivot_df = X_train.pivot(index='DiscussionID', columns='MemberID', values='WeightedInstrumentalnessRating')
pivot_df = pivot_df.fillna(0)

In [ ]:
user = 341
album = 68
user_sims = {}

for member_id in pivot_df.columns:
    if member_id != user:
        print('User:', member_id)
        print('Cosine Sim:', cosine_similarity(pivot_df[user], pivot_df[member_id]))
        user_sims[member_id] = cosine_similarity(pivot_df[user], pivot_df[member_id])

In [53]:
for idx, row in X_test.iterrows():
    user = row['MemberID']
    album = row['DiscussionID']
    user_sims = {}
    
    if user not in pivot_df.columns:
        y_pred.append(int(np.round(X_train['WeightedInstrumentalnessRating'].mean(),0)))
        continue
    if album not in pivot_df.index:
        y_pred.append(int(np.round(X_train['WeightedInstrumentalnessRating'].mean(),0)))
        continue

    for member_id in pivot_df.columns:
        if member_id != user:
            user_sims[member_id] = cosine_similarity(pivot_df[user], pivot_df[member_id])

    rated_users = []
    for i in pivot_df.columns:
        if pivot_df[i][album] != 0 and i != user:
            rated_users.append(i)
    rated_user_sims = []
    for u in rated_users:
        rated_user_sims.append(user_sims[u])

    top5_rated_users = []
    top5_sims = []
    sorted_pairs = sorted(zip(rated_users, rated_user_sims), key=lambda x: x[1], reverse=True)
    for top5_user, sim in sorted_pairs[:5]:
        top5_rated_users.append(top5_user)
        top5_sims.append(sim)
    normalized_top5_sims = [x * sum(top5_sims) for x in normalized_top5_sims]

    pred_r = 0
    ind = 0
    # # predict the rating with the weighted avg
    for u in top5_rated_users:
        pred_r += normalized_top5_sims[ind] * pivot_df[u][album]
        ind += 1

    y_pred.append(int(np.round(pred_r,0)))
    
y_pred = np.array(y_pred)
rmse = np.sqrt(np.mean((y_pred - y_test)**2))
print("RMSE:", rmse)

/var/folders/zg/nhdtkkv14fj5w_ftm8sjrdt00000gn/T/ipykernel_3300/3092099898.py:5: RuntimeWarning: invalid value encountered in scalar divide
  retValue = dot_product / (norm_vec1 * norm_vec2) if not math.isnan(dot_product / (norm_vec1 * norm_vec2)) else 0


RMSE: 2.236919177526273


# Your Turn
Fill out the the database with audio features (watch out for 429 errors!) and run some of the filtering techniques we've covered on it.

Content filtering is a great fit here

also be cognisant of the fact that audio features are numerical values, and thus can be leveraged in weighted sums, numerical analysis, and any number of data science techniques.

and like mentioned in the content filtering slides, feel free to go above and beyond with any external resoruces (like RateYourMusic, AlbumOfTheYear, etc.) you'd like!

In [44]:
# TODO: integrate Spotify with your filtering code